In [1]:
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
pd.set_option('max_colwidth',500)
pd.set_option('display.max_rows',None)

In [2]:
consumerkey="t8OfsUuN8gcrWneZcTr7hPwru"
counsumerSecret="qIxVtHvHGgkHw62hxRYKcFcw0T1vA8J6trYyROsKSKCIbUrTg1"
accessToken="1417371823847575553-kXZCF0AxfUucOIlZeqwIfHSHM300Ec"
accessTokenSecret="YCiq65sDrP66MBebMD6aMnvJZEysanKaJS2QNaDEQXOD4"

auth=tweepy.OAuthHandler(consumerkey,counsumerSecret)
auth.set_access_token(accessToken,accessTokenSecret)
api = tweepy.API(auth)

### FETCHING TWEETS DATA 

In [ ]:
tweet_df=pd.DataFrame(columns=['ID','Text','No. of likes','Retweet count','Date n time'])
tweets = tweepy.Cursor(api.user_timeline, id="JitinPrasada",tweet_mode="extended").items(2000)    
for tweet in tweets:
#print (tweet.id)
    status = api.get_status(tweet.id)
    new_row = {'ID':tweet.id, 'Text':status.text, 'No. of likes':status.favorite_count, "Retweet count":status.retweet_count,'Date n time':status.created_at}
    tweet_df = tweet_df.append(new_row,ignore_index=True)


In [4]:
tweet_df

### CONVERTING HINDI TWEETS IN ENGLISH LANG USING TEXTBLOB LIBRARY

In [ ]:
for i in range (0,len(tweet_df)):
    output=TextBlob(tweet_df['Text'][i])
    lang=output.detect_language()
    try:
        if (lang!='hi'):
            tweet_df["Text_eng"][i]=str(output)
#             print (df["Text_eng"][i])
        else:
            tweet_df["Text_eng"][i]=str(output.translate(to='en'))
#             print (df["Text_eng"][i])
    except:
        print(i)
        continue


### CLEANING DATA

In [5]:
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|@"," ",x)

In [6]:
remove_sp=lambda x:re.sub(' +', ' ', x)

In [8]:
tweet_df["Clean_text"] = tweet_df['Text_eng'].map(rt).map(remove_sp)

In [ ]:
tweet_df["Clean_text"] = tweet_df['Clean_text'].str.lower()

In [ ]:
tweet_df['Clean_text']=tweet_df['Clean_text'].apply(lambda x: x.strip())

In [ ]:
tweet_df['Date']=tweet_df['Date n time'].apply(lambda x: x.date())

In [ ]:
df['Check'] = df['Date'].apply(lambda x:time.strptime(str(x), "%Y-%m-%d") )

In [ ]:
y = time.strptime('2021-06-09', "%Y-%m-%d")

In [ ]:
english_punctuations = string.punctuation

In [ ]:
def cleaning_punctuations(text):
    translator = str.maketrans('', '', english_punctuations)
    return text.translate(translator)

In [ ]:
tweet_df['Tokens']=''

In [ ]:
tweet_df['Tokens'] = tweet_df['Clean_text'].apply(lambda x: cleaning_punctuations(x))

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
tweet_df.reset_index(drop=True,inplace=True)

In [ ]:
for i in range(0,len(df)):
    word_tokens = word_tokenize(df['Clean_text'][i])
    filtered_sentence=''
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence = filtered_sentence+' '+w
    df['Tokens'][i]=filtered_sentence

In [ ]:
df['Tokens'] = df['Tokens'].apply(lambda x:re.sub(' +', ' ', x))

In [ ]:
df['Tokens'] = df['Tokens'].apply(lambda x:x.lower().strip())

#### joined BJP on 2021-06-09

### segregating in bjp and inc

In [ ]:
df_bjp=df[df['Check']>=y]

In [ ]:
df_bjp.reset_index(drop=True,inplace=True)

In [ ]:
df_inc=df[df['Check']<y]

In [ ]:
df_inc.reset_index(drop=True,inplace=True)

#### CREATING WORDCLOUD

In [ ]:
bjp_tokens =df_bjp['Tokens']
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(bjp_tokens))
plt.imshow(wc)
#plt.savefig(r"C:\Users\ad\Downloads\Jitin_Prasad_After_Joining_BJP.png",dpi=300)

In [ ]:
inc_tokens =df_inc['Tokens']
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(inc_tokens))
plt.imshow(wc)
#plt.savefig(r"C:\Users\ad\Downloads\Jitin_Prasad_before_Joining_BJP.png",dpi=300)

### LIKES COMPARISION

In [ ]:
print ("AFTER HE JOINED BJP")
print ("NO. OF LIKES BETWEEN 29thjuly2021-9thjune2021:- ",sum(df_bjp['No. of likes']))#29thjuly2021-9thjune2021

In [ ]:
print ("DURING INC")
print ("NO. OF LIKES BETWEEN 8thjune2021-28thAug2020:- ",sum(df_inc['No. of likes']))

### FINIDING SENTIMENT

In [ ]:
df[['polarity', 'subjectivity']] = df['Clean_text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

In [ ]:
plt.xlim(-1,1)
plt.hist(df['polarity'],bins=10)
plt.show()

In [ ]:
filters = [(df.polarity <0),
         (df.polarity >=0)&(df.polarity<=0.25),
         (df.polarity >0.25)]

In [ ]:
value=['Negative',"Neutral","Positive"]

In [ ]:
df['Sentiment']=np.select(filters,value)

### CREATING WORD CLOUD

In [ ]:
bjp_tokens =df_bjp['Tokens']
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(bjp_tokens))
plt.imshow(wc)

In [ ]:
inc_tokens =df_inc['Tokens']
plt.figure(figsize = (20,20))
wc = WordCloud(max_words = 1000 , width = 1600 , height = 800,
               collocations=False).generate(" ".join(inc_tokens))
plt.imshow(wc)